# Ex.1: Basic charts using the API

This first example shows how to obtain and plot time series data for a particular station.

##### Libraries Used:
- json
- urllib
- bokeh

For this example, we are interested in creating a basic plot of temperature, wind speed, and humidity for a particular station. I'll use KSLC and grab a time series of data from 0700z to 2100z. This query looks like: http://api.mesowest.net/v2/stations/timeseries?&token=demotoken&vars=air_temp,wind_speed,relative_humidity&stid=kslc&start=201603040700&end=201603042100&units=english&timeformat=%H%M


Note the additional parameters we've added to this query. We are specifically requesting the variables wind speed, temperature, and relative humidity to reduce the query response time/size. We also set the units to English and provide a custom time format because we only care about the hour and minute of the observation since it's just one day.

### In python, we use the built-in libraries for working with http requests and json to make the call to the API:

In [58]:
import urllib.request
import json

q = 'http://api.mesowest.net/v2/stations/timeseries?&token=demotoken&vars=air_temp,wind_speed,relative_humidity&stid=kslc&start=201603040700&end=201603042100&units=english&timeformat=%H%M'
# Open the url and read the contents
resp = urllib.request.urlopen(q).read()
# Decode and make the content a JSON object
stationJSON = json.loads(resp.decode('utf-8'))

### When we inspect the dataset, we can see the structure of the JSON response. Our observation data is found in a nested dictionary in the 'STATION' key.


In [59]:
# stationJSON

### Knowing our structure, let's extract the data into some cleaner variables for adding to our chart.

In [60]:
datetimes = stationJSON['STATION'][0]['OBSERVATIONS']['date_time']
temps = stationJSON['STATION'][0]['OBSERVATIONS']['air_temp_set_1']
rh = stationJSON['STATION'][0]['OBSERVATIONS']['relative_humidity_set_1']
wsp = stationJSON['STATION'][0]['OBSERVATIONS']['wind_speed_set_1']

Great! Now that we have some data, let's plot it to some charts. I prefer charts that are a bit interactive and was excited to use Bokeh for this plot. Hopefully, the code should look very similar for those that use matplotlib or another plotting library. 

### Plotting our data is actually pretty simple. 

We import the required functions, set a data source, create some tooltips, and then generate our plots

In [61]:
from bokeh.io import output_notebook, show, gridplot
output_notebook()
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(datetimes=datetimes, temps=temps, rh=rh, wsp=wsp))

# Create some tooltips for data points
hoverTRH = HoverTool(tooltips=[("TIME", "@datetimes"),("TEMP", "@temps"),("RH", "@rh"),('WSP', "@wsp")])
hoverW = HoverTool(tooltips=[("TIME", "@datetimes"),("TEMP", "@temps"),("RH", "@rh"),('WSP', "@wsp")])

# create a new plot and add temp and RH renderers
leftChart = figure(tools=[hoverTRH, 'box_select, lasso_select'], width=500, height=500, title='Temp (F) and RelH (%)')
leftChart.circle('datetimes', 'temps', source=source, size=5, color="#9E676A", alpha=0.5)
leftChart.line('datetimes', 'rh', source=source, line_width=2, line_color="#4F6054", alpha=1)

# create another new plot and add wind speed renderer
rightChart = figure(tools=[hoverW, 'box_select, lasso_select'], width=500, height=500, title='Wind Speed (mph)')
rightChart.circle('datetimes', 'wsp', source=source, color='#526169')
p = gridplot([[leftChart, rightChart]])

# show the charts
show(p)

Loading BokehJS ...